In [1]:
# https://towardsdatascience.com/a-brief-introduction-to-intent-classification-96fda6b1f557
# Classifies into 21 intents


import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Load Dataset

In [2]:
def load_dataset(filename):
    df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])

    print(df.head())
    intent = df["Intent"]
    unique_intent = list(set(intent))
    sentences = list(df["Sentence"])
  
    return (intent, unique_intent, sentences)

In [3]:
intent, unique_intent, sentences = load_dataset("questions_categories.csv")

                                Sentence              Intent
0                 When do classes start?  faq.important_date
1          When does the semester start?  faq.important_date
2      What day does the semester start?  faq.important_date
3     Which day does the semester start?  faq.important_date
4  What date does the semester start on?  faq.important_date


# Get stopwords and punkt

In [4]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Define stemmer

In [5]:
stemmer = LancasterStemmer()

# Data cleaning

In [6]:
def cleaning(sentences):
    words = []
    for s in sentences:
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        w = word_tokenize(clean)
        # lemmatizing
        words.append([i.lower() for i in w])
    return words

In [7]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:3])

119
[['when', 'do', 'classes', 'start'], ['when', 'does', 'the', 'semester', 'start'], ['what', 'day', 'does', 'the', 'semester', 'start']]


# Input encoding

In [8]:
def create_tokenizer( words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    """Create tokenizer
    """
    token = Tokenizer(filters=filters)
    token.fit_on_texts(words)
    return token

def get_max_length(words):
    """Gets max length of a word
    """
    return(len(max(words, key=len)))


In [9]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = get_max_length(cleaned_words)

print("Vocab size = ", vocab_size, " and Maximum length = ", max_length)

Vocab size =  158  and Maximum length =  17


# Output Encoding

In [10]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [11]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [12]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen=max_length, padding="post"))

In [13]:
padded_doc = padding_doc(encoded_doc, max_length)
padded_doc[:5]

array([[ 6,  5, 24, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [ 6, 12,  3, 21, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [ 9, 14, 12,  3, 21, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [66, 14, 12,  3, 21, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [ 9, 94, 12,  3, 21, 17, 46,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]])

In [14]:
print("Shape of padded docs = ", padded_doc.shape)

Shape of padded docs =  (119, 17)


### Tokenizer

In [15]:
# tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters='!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [16]:
output_tokenizer.word_index

{'faq.professor': 1,
 'faq.important_date': 2,
 'class.time': 3,
 'class.location': 4,
 'faq.department': 5,
 'department.time': 6,
 'faq.class': 7,
 'faq.student': 8,
 'faq.employee': 9,
 'class.professor': 10}

### Encode output given intent and tokenizer and reshape

In [17]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [18]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [19]:
encoded_output.shape

(119, 1)

### Create one hot encoding
Example of one hot encoding:
Consider a domain of [ a, e, i, o, u] and an intent of [ a, i, u ]
The one hot encoding is [ 1, 0, 1, 0, 1] for the domain.

In [20]:
def one_hot(encode):
    o = OneHotEncoder(sparse=False)
    return(o.fit_transform(encode))

In [21]:
print(encoded_output)

[[ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [10]
 [10]
 [10]
 [10]
 [10]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 3]
 [ 3]
 [ 3]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 8]
 [ 8]
 [ 5]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 5]
 [ 5]
 [ 8]
 [ 8]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 8]
 [ 8]
 [ 1]]


In [22]:
output_one_hot = one_hot(encoded_output)

In [23]:
output_one_hot.shape

(119, 10)

## Create Model

In [24]:
from sklearn.model_selection import train_test_split

In [25]:

train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)

In [26]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (95, 17) and train_Y = (95, 10)
Shape of val_X = (24, 17) and val_Y = (24, 10)


### Sequential Model
[ conv ] -> [ batch norm ] -> [ relu ]

A sequential model allows you to create models layer-by-layer in a step-by-step fashion

We instantiate the sequential model first, then add each layer one at a time.
Layers:
    
    Embedding:
        Vocab size:
        Input length:
    Bidirectional:
        LSTM:
    Dense: Relu
    Dropout: 0.5
    Dense: Softmax

In [27]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation = "softmax"))
  
  return model

### Compile and give summary of model
#### Compile
A loss function (or objective function, or optimization score function) is one of the two parameters required to compile a model. We use categorical cross entropy to train a CNN to output a probability over the C classes for each image. It is used for multi-class classification. It is also called softmax loss, a softmax activation plus a cross-entropy loss.

Adam is an adaptive learning rate optimization algorithm designed to train deep neural nets. Adam computes individual learning rates for different parameters, using the first and second moments of gradient to adapt the learning rate for each weight of the neural network.



In [28]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 128)           20224     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 291,946
Trainable params: 271,722
Non-trainable params: 20,224
_________________________________________________________________


## Train Model
Uses checkpoint to save best model at each training

In [29]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 95 samples, validate on 24 samples
Epoch 1/100
32/95 [=========>....................] - ETA: 8s - loss: 2.3025 - accuracy: 0.0938
Epoch 00001: val_loss improved from inf to 2.27336, saving model to model.h5
95/95 [==============================] - 5s 57ms/sample - loss: 2.2907 - accuracy: 0.1895 - val_loss: 2.2734 - val_accuracy: 0.2500
Epoch 2/100
32/95 [=========>....................] - ETA: 0s - loss: 2.2256 - accuracy: 0.3438
Epoch 00002: val_loss improved from 2.27336 to 2.24055, saving model to model.h5
95/95 [==============================] - 0s 2ms/sample - loss: 2.2556 - accuracy: 0.2526 - val_loss: 2.2405 - val_accuracy: 0.2500
Epoch 3/100
32/95 [=========>....................] - ETA: 0s - loss: 2.1906 - accuracy: 0.3750
Epoch 00003: val_loss improved from 2.24055 to 2.19928, saving model to model.h5
95/95 [==============================] - 0s 1ms/sample - loss: 2.2113 - accuracy: 0.2947 - val_loss: 2.1993 - val_accuracy: 0.2500
Epoch 4/100
32/95 [=========>.........

64/95 [===================>..........] - ETA: 0s - loss: 1.5014 - accuracy: 0.5156
Epoch 00028: val_loss improved from 1.70642 to 1.69853, saving model to model.h5
95/95 [==============================] - 0s 5ms/sample - loss: 1.6149 - accuracy: 0.4632 - val_loss: 1.6985 - val_accuracy: 0.4583
Epoch 29/100
64/95 [===================>..........] - ETA: 0s - loss: 1.3594 - accuracy: 0.5625
Epoch 00029: val_loss improved from 1.69853 to 1.66906, saving model to model.h5
95/95 [==============================] - 0s 5ms/sample - loss: 1.5055 - accuracy: 0.5368 - val_loss: 1.6691 - val_accuracy: 0.4583
Epoch 30/100
64/95 [===================>..........] - ETA: 0s - loss: 1.5349 - accuracy: 0.5000
Epoch 00030: val_loss did not improve from 1.66906
95/95 [==============================] - 0s 2ms/sample - loss: 1.5064 - accuracy: 0.5053 - val_loss: 1.6945 - val_accuracy: 0.4583
Epoch 31/100
64/95 [===================>..........] - ETA: 0s - loss: 1.3734 - accuracy: 0.5469
Epoch 00031: val_loss i

Epoch 57/100
64/95 [===================>..........] - ETA: 0s - loss: 0.9997 - accuracy: 0.6875
Epoch 00057: val_loss did not improve from 1.31496
95/95 [==============================] - 0s 2ms/sample - loss: 0.9343 - accuracy: 0.6737 - val_loss: 1.4001 - val_accuracy: 0.5000
Epoch 58/100
64/95 [===================>..........] - ETA: 0s - loss: 0.8307 - accuracy: 0.7656
Epoch 00058: val_loss did not improve from 1.31496
95/95 [==============================] - 0s 2ms/sample - loss: 0.8926 - accuracy: 0.7474 - val_loss: 1.4584 - val_accuracy: 0.5833
Epoch 59/100
64/95 [===================>..........] - ETA: 0s - loss: 0.9829 - accuracy: 0.6406
Epoch 00059: val_loss did not improve from 1.31496
95/95 [==============================] - 0s 2ms/sample - loss: 0.9573 - accuracy: 0.6632 - val_loss: 1.5088 - val_accuracy: 0.5417
Epoch 60/100
64/95 [===================>..........] - ETA: 0s - loss: 0.9199 - accuracy: 0.6719
Epoch 00060: val_loss did not improve from 1.31496
95/95 [============

32/95 [=========>....................] - ETA: 0s - loss: 0.5970 - accuracy: 0.7500
Epoch 00086: val_loss did not improve from 1.19794
95/95 [==============================] - 0s 2ms/sample - loss: 0.5108 - accuracy: 0.8526 - val_loss: 1.4737 - val_accuracy: 0.5417
Epoch 87/100
64/95 [===================>..........] - ETA: 0s - loss: 0.3707 - accuracy: 0.9062
Epoch 00087: val_loss did not improve from 1.19794
95/95 [==============================] - 0s 2ms/sample - loss: 0.5053 - accuracy: 0.8421 - val_loss: 1.4349 - val_accuracy: 0.5833
Epoch 88/100
64/95 [===================>..........] - ETA: 0s - loss: 0.5344 - accuracy: 0.8281
Epoch 00088: val_loss did not improve from 1.19794
95/95 [==============================] - 0s 2ms/sample - loss: 0.5460 - accuracy: 0.8000 - val_loss: 1.3151 - val_accuracy: 0.5417
Epoch 89/100
64/95 [===================>..........] - ETA: 0s - loss: 0.4273 - accuracy: 0.8750
Epoch 00089: val_loss did not improve from 1.19794
95/95 [=========================

## Load Model
Loads the best model found from training above, model.h5

In [37]:
model = load_model("model.h5")

### Get predicted probability for a given text

In [31]:

def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

### Get final output for the prediction and the classes of intents

In [32]:

def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))

### Use the model =]

In [33]:

text = "Can you help me?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['can', 'you', 'help', 'me']
commonQ.assist has confidence = 0.32932204
commonQ.query has confidence = 0.139143
contact.contact has confidence = 0.09659231
commonQ.bot has confidence = 0.08079043
faq.biz_new has confidence = 0.077273935
commonQ.name has confidence = 0.07718031
commonQ.how has confidence = 0.069322
faq.bad_service has confidence = 0.059703138
faq.apply_register has confidence = 0.022363504
commonQ.wait has confidence = 0.014274337
commonQ.not_giving has confidence = 0.013744186
faq.aadhaar_missing has confidence = 0.006254018
commonQ.just_details has confidence = 0.0044244975
faq.application_process has confidence = 0.0040695267
faq.borrow_use has confidence = 0.0026477
faq.borrow_limit has confidence = 0.0013174948
faq.biz_simpler has confidence = 0.00070938957
faq.approval_time has confidence = 0.00048008468
faq.biz_category_missing has confidence = 0.00029588217
faq.banking_option_missing has confidence = 6.383373e-05
faq.address_proof has confidence = 2.8412622e-05


In [38]:

text = "How do I apply for this position?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['how', 'do', 'i', 'apply', 'for', 'this', 'position']
faq.application_process has confidence = 0.9228019
faq.apply_register has confidence = 0.07120995
commonQ.assist has confidence = 0.0022965271
contact.contact has confidence = 0.0022584863
faq.borrow_use has confidence = 0.0011869266
faq.biz_simpler has confidence = 7.222695e-05
faq.approval_time has confidence = 3.9839113e-05
faq.biz_new has confidence = 3.6369358e-05
faq.borrow_limit has confidence = 2.3749713e-05
faq.bad_service has confidence = 2.2953378e-05
faq.biz_category_missing has confidence = 2.0747395e-05
faq.banking_option_missing has confidence = 1.7692144e-05
commonQ.how has confidence = 8.244553e-06
commonQ.wait has confidence = 1.4011844e-06
commonQ.query has confidence = 1.3473338e-06
commonQ.name has confidence = 1.0343318e-06
faq.aadhaar_missing has confidence = 3.699744e-07
faq.address_proof has confidence = 1.3429752e-07
commonQ.just_details has confidence = 1.1053147e-07
commonQ.bot has confidence = 7.3278206

In [40]:

text = "Wait for me"
pred = predictions(text)
get_final_output(pred, unique_intent)

['wait', 'for', 'me']
faq.apply_register has confidence = 0.2630215
commonQ.assist has confidence = 0.1846362
commonQ.how has confidence = 0.119165204
commonQ.wait has confidence = 0.112347364
contact.contact has confidence = 0.07797397
commonQ.name has confidence = 0.052337676
commonQ.bot has confidence = 0.045500476
commonQ.query has confidence = 0.030452987
faq.aadhaar_missing has confidence = 0.027099965
faq.biz_new has confidence = 0.01953664
commonQ.not_giving has confidence = 0.015911063
faq.application_process has confidence = 0.01587559
faq.bad_service has confidence = 0.01562931
commonQ.just_details has confidence = 0.012118423
faq.approval_time has confidence = 0.0032076482
faq.borrow_use has confidence = 0.0016377135
faq.banking_option_missing has confidence = 0.0010719093
faq.biz_category_missing has confidence = 0.00094919227
faq.borrow_limit has confidence = 0.0007261337
faq.address_proof has confidence = 0.00048034557
faq.biz_simpler has confidence = 0.00032065253


In [41]:

text = "Who can I call to help me?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['who', 'can', 'i', 'call', 'to', 'help', 'me']
faq.apply_register has confidence = 0.89374787
faq.application_process has confidence = 0.036169574
commonQ.assist has confidence = 0.031955425
contact.contact has confidence = 0.030594548
commonQ.how has confidence = 0.005335092
commonQ.wait has confidence = 0.001275502
commonQ.name has confidence = 0.00029222755
faq.bad_service has confidence = 0.00012833613
faq.biz_new has confidence = 0.00011115512
commonQ.query has confidence = 0.00010855391
faq.approval_time has confidence = 8.3946055e-05
commonQ.just_details has confidence = 5.583135e-05
faq.aadhaar_missing has confidence = 3.8708153e-05
faq.banking_option_missing has confidence = 3.30851e-05
faq.borrow_use has confidence = 2.912586e-05
commonQ.bot has confidence = 1.8081726e-05
faq.biz_category_missing has confidence = 7.941922e-06
commonQ.not_giving has confidence = 7.613979e-06
faq.borrow_limit has confidence = 4.9950563e-06
faq.biz_simpler has confidence = 1.3852948e-06
faq.add